### clinvar missense prediction w/ feature intersection
* only use consistent positions
* only missense clinvar
* use positions w/ mpc **OR** pathogenic fraction
* calc path freq using counts
* total path freq
* total benign freq

In [4]:
import pandas, numpy
from scipy.stats import entropy
import pydot, pydotplus, graphviz
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from sklearn import linear_model, metrics, tree, svm
from sklearn.neural_network import MLPClassifier
from sklearn.externals.six import StringIO
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import ExtraTreesClassifier
from IPython.display import HTML
%matplotlib inline

In [5]:
def calc_path_freq(rows):
    # sum of freqs for path
    df = rows[ (rows.clin_class=='PATHOGENIC') |
                (rows.clin_class=='LIKLEY_PATHOGENIC')]
    l = len(df)
    pathogenic_sum = sum(df['freq'])
    neg = sum(df['neg_fam'])
    if l == 0:
        return 0, 0, -1, 0
    return pathogenic_sum, pathogenic_sum/l, entropy(df['freq']/pathogenic_sum), l

def calc_benign_freq(rows):
    # sum of freqs for 
    df = rows[ (rows.clin_class=='LIKELY_BENIGN') |
                           (rows.clin_class=='BENIGN')]
    benign_sum = sum(df['freq'])
    l = len(df)
    neg = sum(df['neg_fam'])
    if l == 0:
        return 0, 0, -1, 0
    return benign_sum, benign_sum/l, entropy(df['freq']/benign_sum), l

def calc_path_frac(rows):
    pfam = list(rows['pfam'].values)[0]
    pathogenic = len(rows[ (rows.clin_class=='PATHOGENIC') | (rows.clin_class=='LIKLEY_PATHOGENIC')])
    benign = len(rows[ (rows.clin_class=='LIKELY_BENIGN') | (rows.clin_class=='BENIGN')])
    frac = -1
    if pathogenic+benign:
        frac = pathogenic/(pathogenic+benign)
    
    pf, pf_avg, pf_ent, pcount = calc_path_freq(rows)
    bf, bf_avg, bf_ent, bcount = calc_benign_freq(rows)
    
    r = -1
    if bf:
        r = pf/bf
    return pandas.Series([frac, len(rows), pf, pf_avg, pf_ent, pcount, bf, bf_avg, bf_ent, bcount, r],
                         index=['path_frac', 'size',
                                'path_freq', 'p_freq_avg', 'p_freq_ent', 'ps',
                                'benign_freq', 'b_freq_avg', 'b_freq_ent', 'bs',
                                'fRatio'])

def calc_tot_freq_ratio(rows):
    path_sum = calc_path_freq(rows)
    benign_sum = calc_benign_freq(rows)
    return path_sum/benign_sum

dat_file = '../data/interim/EPIv6.eff.dbnsfp.anno.hHack.dat.xls'
df_pre = pandas.read_csv(dat_file, sep='\t').fillna(0)
df_pre.loc[:, 'freq'] = df_pre['pos_fam']/(df_pre['pos_fam']+df_pre['neg_fam'])
df = (df_pre['pfam'].str.split(',', expand=True)
     .stack()
     .reset_index(level=0)
     .set_index('level_0')
     .rename(columns={0:'pfam'})
     .join(df_pre.drop('pfam',1), how='left')
     )
dd = df.groupby('pfam').apply(calc_path_frac)
ff = dd.reset_index()

# mk domain features
def match(row, domain_info):
    ls = []
    for pfam in row['pfam'].split(','):
        if pfam in domain_info:
            if domain_info[pfam][2] == 0:
                ls.append(domain_info[pfam])
    if len(ls) == 0:
        for pfam in row['pfam'].split(','):
            if pfam in domain_info:
                return domain_info[pfam]
        
    if len(ls):
        return ls[0]
    else:
        return (0, 0,
                0, 0, -1, 0,
                0, 0, -1, 0,
                -1, 1)
    
ff.loc[:, 'path_na'] = ff.apply(lambda row: 1 if row['path_frac']==-1 else 0, axis=1)
domain_info = {pfam:[path_frac, size,
                     path_freq, path_avg, path_ent, pc,
                     b_freq, b_avg, b_ent, bc,
                     fr, path_na]
               for pfam, path_frac, size, path_freq, path_avg, path_ent, pc, b_freq, b_avg, b_ent, bc, fr, path_na
               in ff.values}

df_pre.loc[:, 'path_frac_t'] = df_pre.apply(lambda row: match(row, domain_info)[0], axis=1)
df_pre.loc[:, 'size_t'] = df_pre.apply(lambda row: match(row, domain_info)[1], axis=1)
df_pre.loc[:, 'path_na_t'] = df_pre.apply(lambda row: match(row, domain_info)[-1], axis=1)
df_pre.loc[:, 'in_none_pfam'] = df_pre.apply(lambda row: 1 if 'none' in row['pfam'] else 0, axis=1)

# use patient counts
df_pre.loc[:, 'path_freq'] = df_pre.apply(lambda row: match(row, domain_info)[2], axis=1)
df_pre.loc[:, 'path_avg'] = df_pre.apply(lambda row: match(row, domain_info)[3], axis=1)
df_pre.loc[:, 'path_ent'] = df_pre.apply(lambda row: match(row, domain_info)[4], axis=1)
df_pre.loc[:, 'path_cnt'] = df_pre.apply(lambda row: match(row, domain_info)[5], axis=1)

df_pre.loc[:, 'benign_freq'] = df_pre.apply(lambda row: match(row, domain_info)[6], axis=1)
df_pre.loc[:, 'benign_avg'] = df_pre.apply(lambda row: match(row, domain_info)[7], axis=1)
df_pre.loc[:, 'benign_ent'] = df_pre.apply(lambda row: match(row, domain_info)[8], axis=1)
df_pre.loc[:, 'benign_cnt'] = df_pre.apply(lambda row: match(row, domain_info)[9], axis=1)

df_pre.loc[:, 'path_benign_freq_r'] = df_pre.apply(lambda row: match(row, domain_info)[10], axis=1)
#df_pre.loc[:, 'path_na_t'] = df_pre.apply(lambda row: match(row, domain_info)[2], axis=1)

In [12]:
# this is for training
# use not just missense
# I do not need to require an mpc score here anymore (df_pre.mpc>0)
df_x_pre = df_pre[ (df_pre.clin_class != 'VUS') ]
df_s = df_x_pre.groupby('pfam').size().reset_index()
multi_pfam = set( df_s[df_s[0]>1]['pfam'].values )
df_x_pre.loc[:, 'multi_pfam'] = df_x_pre.apply(lambda row: row['pfam'] in multi_pfam, axis=1)
df_x = df_x_pre[ (df_x_pre.multi_pfam) & (df_x_pre.eff=='missense_variant') & (df_x_pre.mpc>0)]
df_x.loc[:, 'y'] = df_x.apply(lambda row: 1 if row['clin_class'] in ('PATHOGENIC', 'LIKLEY_PATHOGENIC')
                            else 0, axis=1)
df_x.head()

/opt/conda/lib/python3.4/site-packages/pandas/core/indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/lib/python3.4/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,chrom,pos,ref,alt,clin_class,pfam,af_1kg_all,eff,pos_fam,neg_fam,...,path_avg,path_ent,path_cnt,benign_freq,benign_avg,benign_ent,benign_cnt,path_benign_freq_r,multi_pfam,y
12,1,40555089,G,C,PATHOGENIC,"Palm_thioest:6,Palm_thioest:7",0.000000,missense_variant,1,8544,...,0.000293,0.500402,2.0,0.000000,0.000000,-1.0,0.0,-1.0,True,1
16,1,40557070,T,A,PATHOGENIC,Palm_thioest:9,0.000599,missense_variant,2,8543,...,0.000176,0.636514,2.0,0.000000,0.000000,-1.0,0.0,-1.0,True,1
18,1,40557754,A,C,PATHOGENIC,Palm_thioest:10,0.000000,missense_variant,1,8544,...,0.000117,0.000000,1.0,0.000234,0.000234,0.0,1.0,0.5,True,1
19,1,40557769,T,C,LIKELY_BENIGN,Palm_thioest:10,0.000000,missense_variant,2,8543,...,0.000117,0.000000,1.0,0.000234,0.000234,0.0,1.0,0.5,True,0
34,1,43393355,C,T,PATHOGENIC,"MFS_1:44,Sugar_tr:29",0.000000,missense_variant,1,8419,...,0.000148,1.332179,4.0,0.000000,0.000000,-1.0,0.0,-1.0,True,1


In [13]:
train_keys = {':'.join([str(x) for x in v]):True for v in df_x[['chrom', 'pos', 'ref', 'alt']].values}

In [14]:
clin_file = '../data/interim/clinvar/clinvar.dat'
#clinvar_df_pre = pandas.read_csv(clin_file, sep='\t').fillna(0)
def calc_final_sig(row):
    sig_set = set(str(row['clinSig'].split('|')))
    has_benign = '2' in sig_set or '3' in sig_set
    has_path = '4' in sig_set or '5' in sig_set
    if has_path and not has_benign:
        return 1
    if not has_path and has_benign:
        return 0
    return -1

# & (clinvar_df_pre.not_in_training)

# need a smarter match to domain here
#m = pandas.merge(clinvar_df, ff, on='pfam', how='left')
#m.head()

In [22]:
def eval_pred(row):
    if (row['tree_pred']>.9 and row['y']==1) or (row['tree_pred']<.1 and row['y']==0):
        return 'right'
    if (row['tree_pred']>.9 and row['y']==0) or (row['tree_pred']<.1 and row['y']==1):
        return 'wrong'
    return 'vus'

In [29]:
def do_it(gene):
    focus_gene_ls = (gene,)
    clinvar_df_pre = pandas.read_csv(clin_file, sep='\t').fillna(0)

    clinvar_df_pre.loc[:, "y"] = clinvar_df_pre.apply(calc_final_sig, axis=1)
    clinvar_df_pre.loc[:, "key"] = clinvar_df_pre.apply(lambda row: ':'.join([str(row[x]) for x in ['chrom', 'pos', 'ref', 'alt']]), axis=1)
    clinvar_df_pre.loc[:, "not_in_training"] = clinvar_df_pre.apply(lambda row: not row['key'] in train_keys, axis=1)
    clinvar_df_pre.loc[:, "is_focus"] = clinvar_df_pre.apply(lambda row: row['gene'] in focus_gene_ls, axis=1)

    clinvar_df = clinvar_df_pre[(clinvar_df_pre.eff=='missense_variant')
                            & (clinvar_df_pre.not_in_training)
                            & (clinvar_df_pre.mpc>0)
                            & (clinvar_df_pre.is_focus)
                            & (clinvar_df_pre.y!=-1) ].drop_duplicates()
    clinvar_df.loc[:, 'path_frac_t'] = clinvar_df.apply(lambda row: match(row, domain_info)[0], axis=1)
    clinvar_df.loc[:, 'size_t'] = clinvar_df.apply(lambda row: match(row, domain_info)[1], axis=1)
    clinvar_df.loc[:, 'path_freq'] = clinvar_df.apply(lambda row: match(row, domain_info)[2], axis=1)
    clinvar_df.loc[:, 'path_avg'] = clinvar_df.apply(lambda row: match(row, domain_info)[3], axis=1)
    clinvar_df.loc[:, 'path_ent'] = clinvar_df.apply(lambda row: match(row, domain_info)[4], axis=1)
    clinvar_df.loc[:, 'path_cnt'] = clinvar_df.apply(lambda row: match(row, domain_info)[5], axis=1)
    clinvar_df.loc[:, 'benign_freq'] = clinvar_df.apply(lambda row: match(row, domain_info)[6], axis=1)
    clinvar_df.loc[:, 'benign_avg'] = clinvar_df.apply(lambda row: match(row, domain_info)[7], axis=1)
    clinvar_df.loc[:, 'benign_ent'] = clinvar_df.apply(lambda row: match(row, domain_info)[8], axis=1)
    clinvar_df.loc[:, 'benign_cnt'] = clinvar_df.apply(lambda row: match(row, domain_info)[9], axis=1)
    clinvar_df.loc[:, 'path_benign_freq_r'] = clinvar_df.apply(lambda row: match(row, domain_info)[10], axis=1)
    clinvar_df.loc[:, 'path_na_t'] = clinvar_df.apply(lambda row: match(row, domain_info)[-1], axis=1)
    clinvar_df.loc[:, 'in_none_pfam'] = clinvar_df.apply(lambda row: 1 if 'none' in row['pfam'] else 0, axis=1)
    
    # train new tree and apply to clinvar
    forest = ExtraTreesClassifier(n_estimators=300,
                                  random_state=13,
                                  bootstrap=True,
                                  max_features=7,
                                  min_samples_split=2,
                                  max_depth=8,
                                  min_samples_leaf=5,
                                  n_jobs=4)

    all_preds = []
    all_truth = []

    cols = ['mpc', 'size_t', 'path_frac_t', 'in_none_pfam',
        'path_freq', 'path_avg', 'path_ent', 'path_cnt',
        'benign_freq', 'benign_avg', 'benign_ent', 'benign_cnt',
        'af_1kg_all', 'mtr', 'path_benign_freq_r']
    X, y = df_x[cols], df_x['y']
    forest.fit(X, y)

    X_clin, y_clin = clinvar_df[cols], clinvar_df['y']
    preds = [ x[1] for x in forest.predict_proba(X_clin) ]
    clinvar_df['tree_pred'] = preds
    clinvar_df.loc[:, 'PredictionStatus'] = clinvar_df.apply(eval_pred, axis=1)
    fpr_tree, tpr_tree, _ = metrics.roc_curve(y_clin, preds, pos_label=1)
    tree_auc = metrics.auc(fpr_tree, tpr_tree)
    print(gene, tree_auc)
    
    g_df = (clinvar_df[['gene', 'chrom', 'pos', 'ref', 'alt', 'PredictionStatus']]
        .groupby(['gene','PredictionStatus'])
        .size().reset_index().rename(columns={0:'size'}))
    dd = g_df.groupby('gene').sum().reset_index()
    use_genes = set(dd[dd['size']>0]['gene'].values)
    g_df.loc[:, 'keep'] = g_df.apply(lambda row: row['gene'] in use_genes, axis=1)
    sns.set(font_scale=1.75)
    flatui = ["#2ecc71",  "#3498db",  "#e74c3c",]
    ss = sns.factorplot(x='gene', hue='PredictionStatus', y='size', data=g_df[g_df['keep']],
                        kind='bar', palette=sns.color_palette(flatui), size=5, aspect=3)
    ss.set_ylabels('ClinVar missense variants')
    ss.set_xlabels('')
    ss.savefig("../docs/plots/clinvar_%s_eval.png" % (gene,))

    # train new tree and apply to clinvar: just pathogenic frac
    #tree_clf = linear_model.LogisticRegression(penalty='l1', fit_intercept=True)
    #poly = PolynomialFeatures(degree=6, interaction_only=False, include_bias=False)

    all_preds = []
    all_truth = []
    cols = ['size_t', 'path_frac_t', 'in_none_pfam',
        'path_freq', 'path_avg', 'path_ent', 'path_cnt',
        'benign_freq', 'benign_avg', 'benign_ent', 'benign_cnt',
        'af_1kg_all', 'mtr', 'path_benign_freq_r']
    X, y = df_x[cols], df_x['y']
    forest.fit(X, y)
    
   # cols =  ['size_t', 'path_na_t', 'path_frac_t', 'in_none_pfam','path_freq', 'path_avg', 'path_ent',
   #     'benign_freq', 'benign_avg', 'benign_ent',
   #     'af_1kg_all', 'mtr', 'path_benign_freq_r'] #['size_t', 'path_na_t', 'path_frac_t', 'path_freq', 'benign_freq', 'in_none_pfam',]
    #X, y = poly.fit_transform(df_x[cols]), df_x['y'] #X, y = df_x[cols], df_x['y']
    #tree_clf.fit(X, y)

    X_clin, y_clin = clinvar_df[cols], clinvar_df['y'] #X_clin, y_clin = poly.fit_transform(clinvar_df[cols]), clinvar_df['y'] #clinvar_df[cols], clinvar_df['y']
    preds = [ x[1] for x in forest.predict_proba(X_clin) ]
    fpr_tree_nm, tpr_tree_nm, _ = metrics.roc_curve(y_clin, preds, pos_label=1)
    tree_auc_nm = metrics.auc(fpr_tree_nm, tpr_tree_nm)
    
    scores = clinvar_df['mpc'].values
    truth = clinvar_df['y'].values
    fpr_mpc, tpr_mpc, _ = metrics.roc_curve(truth, scores, pos_label=1)
    mpc_auc = metrics.auc(fpr_mpc, tpr_mpc)
    plt.clf()
    sns.set(font_scale=1.5)
    plt.plot(fpr_tree, tpr_tree, label='Domain Burden + MPC (%.2f)' % (tree_auc,), color='green')
    plt.plot(fpr_tree_nm, tpr_tree_nm, label='Domain Burden (%.2f)' % (tree_auc_nm,), color='orange')
    plt.plot(fpr_mpc, tpr_mpc, label='MPC (%.2f)' % (mpc_auc,), color='black')
    plt.legend(loc=4)
    plt.title('ClinVar %s (w/o GeneDx) missense variant ROC' % (gene,))
    plt.savefig("../docs/plots/clinvar_%s_roc.png" % (gene,))
    plt.clf()

In [30]:
for gene in ('SCN1A','SCN2A','KCNQ2', 'KCNQ3', 'CDKL5',
             'PCDH19', 'SCN1B', 'SCN8A', 'SLC2A1', 'SPTAN1', 'STXBP1', 'TSC1'):
    do_it(gene)

SCN1A 0.94094488189
SCN2A 0.9375
KCNQ2 1.0
KCNQ3 0.875
CDKL5 0.863636363636
PCDH19 0.96
SCN1B 0.86
SCN8A 1.0
SLC2A1 0.375
SPTAN1 0.546666666667
STXBP1 0.857142857143
TSC1 0.766666666667
